In [54]:
from cv2 import *
import joblib
import os
import csv
import numpy as np
import sklearn
from sklearn import svm
import time
from random import randint

import pandas as pd 

In [49]:
os.getcwd()
os.chdir('/Users/yueyingteng/Documents/2016.9/Big Data /kaggle')

In [3]:
# folderPath='./fish/ALB'
# # ALB_index = set(range(2544))
# image_name = os.listdir(folderPath)
# image_names = image_name[0:len(image_name)-1]
# # image_names=[image_names[i] for i in ALB_index]
# image_paths = [os.path.join(folderPath, f) for f in image_names]
# for image_path in image_paths:
#         print image_path, type(image_path)

In [235]:
# image_paths
folders = [f for f in os.listdir('./fish') if not f.startswith('.')]
image_paths0 = []
nums = []
for folder in folders:
    image_name = os.listdir(os.path.join('./fish',folder))
    image_names = image_name[0:len(image_name)-1]
    image_paths0.append([os.path.join(os.path.join('./fish',folder), f) for f in image_names])
    nums.append((len(image_name)-1))
image_paths = [item for sublist in image_paths0 for item in sublist]

In [5]:
# # [float(i) for i in lst]
# # len(image_paths)
# # sum(nums)
# for image_path in image_paths:
#         print image_path, type(image_path)

In [236]:
#image_classes
image_classes = []
for i in range(len(nums)):
    image_classes.append(np.full((1,nums[i]),i))
image_classes = np.concatenate(image_classes, axis = 1)

In [237]:
a = np.ravel(image_classes)
image_classes = a.tolist()
image_classes_train = image_classes[0:len(image_classes)]

In [238]:
image_classes_train[4000]

5.0

In [ ]:
# image_classes=image_classes[0:len(image_classes)-1]
# index=set(range(len(image_classes)))
# index=index.difference(corrupt)
# image_classes=[image_classes[i] for i in index]

In [60]:
# a = np.full((1,5),0)
# b = np.full((1,6), 1)
# c = np.concatenate((a, b,d), axis=1)
# c

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.]])

In [239]:
type(image_paths),len(image_paths), type(image_classes), len(image_classes)

(list, 4091, list, 4091)

In [241]:
# feature_det = cv2.FeatureDetector_create("SIFT")
feature_det = xfeatures2d.SIFT_create()

def preProcessImages(image_paths):
    descriptors= []
    for image_path in image_paths:
        im = imread(image_path)
        kpts = feature_det.detect(im)
        # kpts, des = descr_ext.compute(im, kpts)
        kpts, des = feature_det.compute(im, kpts)
        descriptors.append(des)
    return descriptors

In [242]:
start = time.time()
descriptors= preProcessImages(image_paths)
end = time.time()
print "minutes spent in Descriptors"
print (end - start)/60

minutes spent in Descriptors
2.52184234858


In [243]:
len(descriptors)

4091

In [247]:
descriptors_none=[]
for i, j in enumerate(descriptors):
    if j == None:
        descriptors_none.append(i)

descriptors=[i for i in descriptors if i!= None]
image_classes_train=[image_classes_train[i] for i in range(len(image_classes_train)) if i not in descriptors_none]
image_paths=[image_paths[i] for i in range(len(image_paths)) if i not in descriptors_none]


/Users/yueyingteng/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  app.launch_new_instance()
/Users/yueyingteng/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [248]:
len(image_classes_train), len(image_paths), len(descriptors_none)

(4089, 4089, 0)

In [81]:
paths = pd.DataFrame(image_paths, columns = ['paths'])
classes = pd.DataFrame(image_classes_train, columns = ['classes'])

In [228]:
paths.shape, classes.shape

((4089, 1), (4089, 1))

In [102]:
fishes = classes.join(paths)
# backup save to csv 
fishes.to_csv('fishes.csv', index = False)

In [108]:
fishes.groupby('classes').count()

,paths
classes,
0.0,2543
1.0,306
2.0,123
3.0,100
4.0,195
5.0,822


0

In [59]:
matcher = BFMatcher(NORM_L2)

# extract descriptor of the new images baesed on the constructed vocabulary
# bow_extract  =cv2.BOWImgDescriptorExtractor(descr_ext,matcher)
bow_extract  = BOWImgDescriptorExtractor(feature_det,matcher)

def getImagedata(feature_det,bow_extract,path):
    im = imread(path)
    featureset = bow_extract.compute(im, feature_det.detect(im))
    return featureset
# returned featureset contains normzlized histogram

In [60]:
# clustering k=500
bow_train = BOWKMeansTrainer(50)

# create the vocabulary
for des in descriptors:
    bow_train.add(des)

In [61]:
start = time.time()
voc = bow_train.cluster()
bow_extract.setVocabulary(voc)

end = time.time()
print "minutes spent in creating Vocabulary"
print (end - start)/60

minutes spent in creating Vocabulary
12.0859995325


In [172]:
# get features from the training data based on the vocabulary & approximate nearest neighbour
# features are used as the training data 
traindata = []  

start = time.time()
for path in image_paths:
    featureset = getImagedata(feature_det,bow_extract,path)
    traindata.append(featureset)

end = time.time()
print ("minutes spent in Extracting vocabulary")
print ((end - start)/60)

minutes spent in Extracting vocabulary
2.41215503216


In [128]:
# from sklearn.datasets import load_digits
# digits = load_digits()
# traindata = digits.data
# image_classes_train = digits.target

In [246]:
print len(traindata)
print len(np.array(image_classes_train))
traindata[0].shape

4089
4089


(1, 50)

In [249]:
a = np.array(traindata)
print a.shape
len(a)

size = len(a)
twodim = a.reshape(size, -1)
print twodim.shape

(4089, 1, 50)
(4089, 50)


In [250]:
s = len(np.array(image_classes_train))
l = np.array(image_classes_train).reshape(s, -1)
print l.shape, np.array(image_classes_train).shape

(4089, 1) (4089,)


In [232]:
np.array(image_classes_train).shape

(4091,)

In [251]:
start = time.time()
lin_clf = svm.LinearSVC()
lin_clf.fit(twodim, np.array(image_classes_train)) 

joblib.dump((voc,lin_clf), "imagereco.pkl", compress=3)

end = time.time()
print (end - start)/60

0.00256333351135


In [254]:
start = time.time()

clf = svm.SVC(decision_function_shape='ovo')
clf.fit(twodim, np.array(image_classes_train)) 

# dec = clf.decision_function([[1]])
# dec.shape[1] # 4 classes: 4*3/2 = 6

# clf.decision_function_shape = "ovr"
# dec = clf.decision_function([[1]])
# dec.shape[1] # 4 classes

joblib.dump((voc,clf), "imagereco.pkl", compress=3)

end = time.time()
print (end - start)/60
# LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
#      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
#      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
#      verbose=0)
# dec = lin_clf.decision_function([[1]])
# dec.shape[1]


0.0165712833405


In [255]:
voc,clf = joblib.load("imagereco.pkl")

In [ ]:
# train linear svm
clf=sklearn.svm.classes.LinearSVC()

# clf = LinearSVC()
clf.fit(traindata, numpy.array(image_classes_train))
joblib.dump((voc,clf), "imagereco.pkl", compress=3)

# model=train(descriptors,image_classes_train,image_paths)
voc, clf = joblib.load("imagereco.pkl")


X=numpy.matrix(traindata)
numpy.savetxt('X2.csv', X,delimiter=',')

my_file = open("label_2_2.txt", "w")
for item in image_classes_train:
    my_file.write("%s\n" % item)

my_file.close()

my_file = open("animal_names_2.txt", "w")
for item in training_names:
    my_file.write("%s\n" % item)

my_file.close()

In [257]:
# Accuracy Training data
prediction = clf.predict(twodim)
p=np.array(prediction)
c=np.array(image_classes_train)

print("Accuracy with training")
np.sum(p==c)/(len(prediction)*1.000)


Accuracy with training


0.62191244803130352

In [260]:
# 2543/4089 = 0.6219

0.0
5.0


TypeError: You have to supply one of 'by' and 'level'

In [ ]:
# Accuracy Test data
prediction_test=[]
for imagepath in testing_paths:
    featureset = getImagedata(feature_det,bow_extract,imagepath)
    prediction_temp = clf.predict(featureset)
    if list(prediction_temp) ==['1']:
        prediction_test.append('1')
    else:
        prediction_test.append('0')


p=numpy.array(prediction_test)
c=numpy.array(image_classes_test)
print("Accuracy with training")
numpy.sum(p==c)/(len(prediction_test)*1.000)